In [102]:
#data manipulation & storage
import pandas as pd
import numpy as np
#visualization
import seaborn as sb
from matplotlib import pyplot as plt
#performance and preprocessing
from sklearn import metrics, model_selection, preprocessing
from sklearn.utils import resample
#models
from sklearn import ensemble, tree, neighbors, naive_bayes, linear_model

In [103]:
def get_means(results_dictionary):
    '''
    Returns average of all keys inside dictionary. 
    results_dictionary must have keys with numeric values
    '''
    if type(results_dictionary) != dict: return None
    average_dict = {}
    for key, val in results_dictionary.items():
        key += ' average'
        average_dict[key] = val.mean()
    return average_dict

In [104]:
def print_dictionary_elts(dictionary):
    '''
    Pretty print key/value pairs of a dictionary
    '''
    for key,val in dictionary.items():
        print(f'{key}: {val}')

In [105]:
def run_model(model, x, y, score, cv):
    '''Run cross-validation on sklearn model
    Arguments:
        model - model to be run using cross-validation
        x - vector of independent variables/features
        y - vector of dependent variables/outcomes
        score - scoring metric(s) to estimate model performance
        cv - sklearn KFold cross-validation object
    Returns:
        dictionary with scoring metric summary based on specified cross-validation
    '''
    return model_selection.cross_validate(
        model,
        x,
        y,
        cv=cv,
        scoring=score)

In [106]:
cv = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)

In [107]:
#scoring = ('accuracy', 'f1_weighted', 'roc_auc_ovo_weighted')

In [108]:
scoring = ('f1_weighted', 'accuracy')

In [109]:
df = pd.read_csv('merged_info_volcanoes.csv')

In [110]:
df

Volcano Number  Volcano Name  VEI   Eruption Category  Latitude  \
0              264050  Sangeang Api  NaN  Confirmed Eruption    -8.200   
1              264050  Sangeang Api  2.0  Confirmed Eruption    -8.200   
2              264050  Sangeang Api  3.0  Confirmed Eruption    -8.200   
3              264050  Sangeang Api  NaN  Uncertain Eruption    -8.200   
4              264050  Sangeang Api  2.0  Confirmed Eruption    -8.200   
...               ...           ...  ...                 ...       ...   
11109          327812      Red Hill  NaN  Confirmed Eruption    34.250   
11110          327812      Red Hill  NaN  Confirmed Eruption    34.250   
11111          327812      Red Hill  NaN  Confirmed Eruption    34.250   
11112          283141     Nantaisan  NaN  Confirmed Eruption    36.765   
11113          222161  Igwisi Hills  1.0  Confirmed Eruption    -4.889   

       Longitude        Country Recoded Volcano Type                   Region  \
0        119.070      Indonesia              Complex                Indonesia   
1        119.070      Indonesia              Complex                Indonesia   
2        119.070      Indonesia              Complex                Indonesia   
3        119.070      Indonesia              Complex                Indonesia   
4        119.070      Indonesia              Complex                Indonesia   
...          ...            ...                  ...                      ...   
11109   -108.830  United States       Volcanic field   Canada and Western USA   
11110   -108.830  United States       Volcanic field   Canada and Western USA   
11111   -108.830  United States       Volcanic field   Canada and Western USA   
11112    139.491          Japan        Stratovolcano  Japan, Taiwan, Marianas   
11113     31.933       Tanzania     Pyroclastic cone       Africa and Red Sea   

             Recoded Dominant Rock Type  \
0      Trachybasalt / Tephrite Basanite   
1      Trachybasalt / Tephrite Basanite   
2      Trachybasalt / Tephrite Basanite   
3      Trachybasalt / Tephrite Basanite   
4      Trachybasalt / Tephrite Basanite   
...                                 ...   
11109             Basalt / Picro-Basalt   
11110             Basalt / Picro-Basalt   
11111             Basalt / Picro-Basalt   
11112      Andesite / Basaltic Andesite   
11113                           Foidite   

                                   Tectonic Setting  Elevation (m)  
0      Subduction zone / Continental crust (>25 km)           1912  
1      Subduction zone / Continental crust (>25 km)           1912  
2      Subduction zone / Continental crust (>25 km)           1912  
3      Subduction zone / Continental crust (>25 km)           1912  
4      Subduction zone / Continental crust (>25 km)           1912  
...                                             ...            ...  
11109        Rift zone / Continental crust (>25 km)           2300  
11110        Rift zone / Continental crust (>25 km)           2300  
11111        Rift zone / Continental crust (>25 km)           2300  
11112  Subduction zone / Continental crust (>25 km)           2486  
11113        Rift zone / Continental crust (>25 km)           1146  

[11114 rows x 12 columns]

In [111]:
df['VEI'].value_counts()

2.0    4012
1.0    1383
3.0    1157
0.0     987
4.0     508
5.0     177
6.0      52
7.0       7
Name: VEI, dtype: int64

In [112]:
null_count =  df.VEI.isnull().sum()
print(f'There are {null_count} observations missing VEI.')

There are 2831 observations missing VEI.


In [113]:
#drop observations with null VEI
bad_indices, vei_values, idx = [], df.VEI.values, 0
for val in vei_values:
    if np.isnan(val): bad_indices.append(idx)
    idx += 1
#print(len(bad_indices))
df.drop(index=bad_indices, inplace=True)

In [114]:
df.shape

(8283, 12)

In [115]:
null_count =  df.VEI.isnull().sum()
print(f'There are {null_count} observations missing VEI.')

There are 0 observations missing VEI.


In [116]:
ind_cols = [
    #'Volcano Number',
    'Latitude',
    'Longitude',
    'Country',
    'Recoded Volcano Type',
    'Region',
    'Recoded Dominant Rock Type',
    'Tectonic Setting',
    'Elevation (m)'
]

In [117]:
dep_cols = [
    #'Volcano Number',
    'VEI'
]

In [118]:
scaler = preprocessing.MinMaxScaler()

In [119]:
# Get indendent variable columns
x = df[ind_cols]
x.shape

(8283, 8)

In [120]:
elevation = x['Elevation (m)'].values
x['Elevation'] = scaler.fit_transform(elevation.reshape(-1, 1))

<ipython-input-120-ef547e4caa2c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Elevation'] = scaler.fit_transform(elevation.reshape(-1, 1))


In [121]:
latitude = x['Latitude'].values
x['Latitude'] = scaler.fit_transform(latitude.reshape(-1,1))

<ipython-input-121-23efc708c28c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Latitude'] = scaler.fit_transform(latitude.reshape(-1,1))


In [122]:
longitude = x['Longitude'].values
x['Longitude'] = scaler.fit_transform(longitude.reshape(-1,1))

<ipython-input-122-c46fa3ce87d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Longitude'] = scaler.fit_transform(longitude.reshape(-1,1))


In [123]:
x= x.drop(['Elevation (m)'], axis=1)
x

Latitude  Longitude        Country Recoded Volcano Type  \
1      0.424978   0.831706      Indonesia              Complex   
2      0.424978   0.831706      Indonesia              Complex   
4      0.424978   0.831706      Indonesia              Complex   
5      0.424978   0.831706      Indonesia              Complex   
6      0.424978   0.831706      Indonesia              Complex   
...         ...        ...            ...                  ...   
11103  0.798281   0.940759         Russia        Stratovolcano   
11104  0.780811   0.158448         Canada        Stratovolcano   
11107  0.009072   0.122292     Antarctica               Shield   
11108  0.739944   0.182500  United States               Shield   
11113  0.445273   0.589356       Tanzania     Pyroclastic cone   

                            Region        Recoded Dominant Rock Type  \
1                        Indonesia  Trachybasalt / Tephrite Basanite   
2                        Indonesia  Trachybasalt / Tephrite Basanite   
4                        Indonesia  Trachybasalt / Tephrite Basanite   
5                        Indonesia  Trachybasalt / Tephrite Basanite   
6                        Indonesia  Trachybasalt / Tephrite Basanite   
...                            ...                               ...   
11103  Kamchatka and Mainland Asia      Andesite / Basaltic Andesite   
11104       Canada and Western USA                            Dacite   
11107                   Antarctica           Trachyte / Trachydacite   
11108       Canada and Western USA             Basalt / Picro-Basalt   
11113           Africa and Red Sea                           Foidite   

                                   Tectonic Setting  Elevation  
1      Subduction zone / Continental crust (>25 km)   0.605136  
2      Subduction zone / Continental crust (>25 km)   0.605136  
4      Subduction zone / Continental crust (>25 km)   0.605136  
5      Subduction zone / Continental crust (>25 km)   0.605136  
6      Subduction zone / Continental crust (>25 km)   0.605136  
...                                             ...        ...  
11103  Subduction zone / Continental crust (>25 km)   0.625884  
11104  Subduction zone / Continental crust (>25 km)   0.666031  
11107       Intraplate / Continental crust (>25 km)   0.729629  
11108        Rift zone / Continental crust (>25 km)   0.570634  
11113        Rift zone / Continental crust (>25 km)   0.544240  

[8283 rows x 8 columns]

In [124]:
x = pd.get_dummies(x)
x

Latitude  Longitude  Elevation  Country_Antarctica  Country_Argentina  \
1      0.424978   0.831706   0.605136                   0                  0   
2      0.424978   0.831706   0.605136                   0                  0   
4      0.424978   0.831706   0.605136                   0                  0   
5      0.424978   0.831706   0.605136                   0                  0   
6      0.424978   0.831706   0.605136                   0                  0   
...         ...        ...        ...                 ...                ...   
11103  0.798281   0.940759   0.625884                   0                  0   
11104  0.780811   0.158448   0.666031                   0                  0   
11107  0.009072   0.122292   0.729629                   1                  0   
11108  0.739944   0.182500   0.570634                   0                  0   
11113  0.445273   0.589356   0.544240                   0                  0   

       Country_Armenia  Country_Australia  Country_Burma (Myanmar)  \
1                    0                  0                        0   
2                    0                  0                        0   
4                    0                  0                        0   
5                    0                  0                        0   
6                    0                  0                        0   
...                ...                ...                      ...   
11103                0                  0                        0   
11104                0                  0                        0   
11107                0                  0                        0   
11108                0                  0                        0   
11113                0                  0                        0   

       Country_Cameroon  Country_Canada  ...  \
1                     0               0  ...   
2                     0               0  ...   
4                     0               0  ...   
5                     0               0  ...   
6                     0               0  ...   
...                 ...             ...  ...   
11103                 0               0  ...   
11104                 0               1  ...   
11107                 0               0  ...   
11108                 0               0  ...   
11113                 0               0  ...   

       Tectonic Setting_Intraplate / Continental crust (>25 km)  \
1                                                      0          
2                                                      0          
4                                                      0          
5                                                      0          
6                                                      0          
...                                                  ...          
11103                                                  0          
11104                                                  0          
11107                                                  1          
11108                                                  0          
11113                                                  0          

       Tectonic Setting_Intraplate / Intermediate crust (15-25 km)  \
1                                                      0             
2                                                      0             
4                                                      0             
5                                                      0             
6                                                      0             
...                                                  ...             
11103                                                  0             
11104                                                  0             
11107                                                  0             
11108                                                  0             
11113                                                

In [125]:
x.describe()

Latitude    Longitude    Elevation  Country_Antarctica  \
count  8283.000000  8283.000000  8283.000000         8283.000000   
mean      0.560899     0.613501     0.622253            0.003622   
std       0.183284     0.317051     0.112818            0.060077   
min       0.000000     0.000000     0.000000            0.000000   
25%       0.429023     0.291047     0.550282            0.000000   
50%       0.556952     0.798089     0.605136            0.000000   
75%       0.706629     0.888600     0.684474            0.000000   
max       1.000000     1.000000     1.000000            1.000000   

       Country_Argentina  Country_Armenia  Country_Australia  \
count        8283.000000      8283.000000        8283.000000   
mean            0.000483         0.000121           0.002052   
std             0.021971         0.010988           0.045260   
min             0.000000         0.000000           0.000000   
25%             0.000000         0.000000           0.000000   
50%             0.000000         0.000000           0.000000   
75%             0.000000         0.000000           0.000000   
max             1.000000         1.000000           1.000000   

       Country_Burma (Myanmar)  Country_Cameroon  Country_Canada  ...  \
count              8283.000000       8283.000000     8283.000000  ...   
mean                  0.000121          0.002294        0.000724  ...   
std                   0.010988          0.047842        0.026906  ...   
min                   0.000000          0.000000        0.000000  ...   
25%                   0.000000          0.000000        0.000000  ...   
50%                   0.000000          0.000000        0.000000  ...   
75%                   0.000000          0.000000        0.000000  ...   
max                   1.000000          1.000000        1.000000  ...   

       Tectonic Setting_Intraplate / Continental crust (>25 km)  \
count                                        8283.000000          
mean                                            0.010986          
std                                             0.104245          
min                                             0.000000          
25%                                             0.000000          
50%                                             0.000000          
75%                                             0.000000          
max                                             1.000000          

       Tectonic Setting_Intraplate / Intermediate crust (15-25 km)  \
count                                        8283.000000             
mean                                            0.001328             
std                                             0.036420             
min                                             0.000000             
25%                                             0.000000             
50%                                             0.000000             
75%                                             0.000000             
max                                             1.000000             

       Tectonic Setting_Intraplate / Oceanic crust (< 15 km)  \
count                                        8283.000000       
mean                                            0.058674       
std                                             0.235028       
min                                             0.000000       
25%                                             0.000000       
50%                                             0.000000       
75%                                             0.000000       
max                                             1.000000       

       Tectonic Setting_Rift zone / Continental crust (>25 km)  \
count                                        8283.000000         
mean                                            0.018592         
std                                             0.135088         
min                                             0.000000         
25%                 

In [126]:
# Get dependent variable columns
y = df[dep_cols]
y.shape

(8283, 1)

In [127]:
# split data into train and validation sets.  Validation will be held out to estimate model generalizability
x_train, x_val, y_train, y_val = \
    model_selection.train_test_split(
        x,
        y,
        test_size = 0.30,
        random_state = 42) 

In [128]:
x_train.shape

(5798, 125)

In [129]:
y_train.shape

(5798, 1)

In [130]:
x_val.shape

(2485, 125)

In [131]:
y_val.shape

(2485, 1)

In [132]:
y_train.value_counts()

VEI
2.0    2782
1.0     966
3.0     804
0.0     712
4.0     360
5.0     129
6.0      39
7.0       6
dtype: int64

In [133]:
y_val.value_counts()

VEI
2.0    1230
1.0     417
3.0     353
0.0     275
4.0     148
5.0      48
6.0      13
7.0       1
dtype: int64

In [134]:
np.ravel(y_train)

array([2., 3., 1., ..., 2., 2., 4.])

In [135]:
# Bring training data back together
training_data = pd.concat([x_train, y_train], axis=1)
training_data

Latitude  Longitude  Elevation  Country_Antarctica  Country_Argentina  \
3920  0.422881   0.842011   0.588521                   0                  0   
9326  0.432480   0.859725   0.511010                   0                  0   
7101  0.795921   0.020748   0.536927                   0                  0   
7849  0.546225   0.618662   0.476826                   0                  0   
9079  0.212893   0.299054   0.648064                   0                  0   
...        ...        ...        ...                 ...                ...   
6977  0.697305   0.883207   0.648303                   0                  0   
6178  0.563958   0.247782   0.752286                   0                  0   
6431  0.867444   0.445846   0.571588                   0                  0   
958   0.243193   0.302564   0.687893                   0                  0   
9060  0.559428   0.252193   0.603625                   0                  0   

      Country_Armenia  Country_Australia  Country_Burma (Myanmar)  \
3920                0                  0                        0   
9326                0                  0                        0   
7101                0                  0                        0   
7849                0                  0                        0   
9079                0                  0                        0   
...               ...                ...                      ...   
6977                0                  0                        0   
6178                0                  0                        0   
6431                0                  0                        0   
958                 0                  0                        0   
9060                0                  0                        0   

      Country_Cameroon  Country_Canada  ...  \
3920                 0               0  ...   
9326                 0               0  ...   
7101                 0               0  ...   
7849                 0               0  ...   
9079                 0               0  ...   
...                ...             ...  ...   
6977                 0               0  ...   
6178                 0               0  ...   
6431                 0               0  ...   
958                  0               0  ...   
9060                 0               0  ...   

      Tectonic Setting_Intraplate / Intermediate crust (15-25 km)  \
3920                                                  0             
9326                                                  0             
7101                                                  0             
7849                                                  0             
9079                                                  0             
...                                                 ...             
6977                                                  0             
6178                                                  0             
6431                                                  0             
958                                                   0             
9060                                                  0             

      Tectonic Setting_Intraplate / Oceanic crust (< 15 km)  \
3920                                                  0       
9326                                                  0       
7101                                                  0       
7849                                                  0       
9079                                                  0       
...                                                 ...       
6977                                                  0       
6178                                                  0       
6431                                                  0       
958                                                   0       
9060                                                  0       

      Tectonic Setting_Rift zone / Continental crust (>25 km)  \
3920        

In [136]:
y_train.shape

(5798, 1)

In [137]:
x_train.shape

(5798, 125)

In [138]:
## Now, construct models

In [139]:
# Figure out the best K for KNN
knn_cv = model_selection.KFold(n_splits=5, random_state=42, shuffle=True)
best_n, best_acc = None, 0
for n in range(40):
    n += 1
    knn = neighbors.KNeighborsClassifier(n_neighbors = n)
    results = get_means(run_model(knn, x_train, np.ravel(y_train), scoring, knn_cv))
    acc = results['test_accuracy average']
    print(f'Number of neighbors is {n}... & accuracy is {acc}')
    if acc > best_acc:
        best_n, best_acc = n, acc

Number of neighbors is 1... & accuracy is 0.48205587456487464
Number of neighbors is 2... & accuracy is 0.45894335187884916
Number of neighbors is 3... & accuracy is 0.4906804320014281
Number of neighbors is 4... & accuracy is 0.5313868971467675
Number of neighbors is 5... & accuracy is 0.5455283984409866
Number of neighbors is 6... & accuracy is 0.5406978370176431
Number of neighbors is 7... & accuracy is 0.552254619023534
Number of neighbors is 8... & accuracy is 0.5472537264585998
Number of neighbors is 9... & accuracy is 0.5538087233346225
Number of neighbors is 10... & accuracy is 0.552946059325816
Number of neighbors is 11... & accuracy is 0.5550147273214125
Number of neighbors is 12... & accuracy is 0.5576019755437208
Number of neighbors is 13... & accuracy is 0.5601884799619172
Number of neighbors is 14... & accuracy is 0.5608779863735087
Number of neighbors is 15... & accuracy is 0.5584667222040404
Number of neighbors is 16... & accuracy is 0.5641568236589213
Number of neighbo

In [140]:
print(f'Best n is {best_n} and best accuracy is {best_acc}')

Best n is 17 and best accuracy is 0.564846925113802


In [141]:
lr = linear_model.LogisticRegression(
    penalty='l2',
    C=0.5,
    random_state=42,
    multi_class='multinomial',
    max_iter = 1000
)
lr

LogisticRegression(C=0.5, max_iter=1000, multi_class='multinomial',
                   random_state=42)

In [142]:
nb = naive_bayes.MultinomialNB(
    alpha=5.0,
    fit_prior=True)
nb

MultinomialNB(alpha=5.0)

In [143]:
knn = neighbors.KNeighborsClassifier(n_neighbors = best_n)
knn

KNeighborsClassifier(n_neighbors=17)

In [144]:
tr = tree.DecisionTreeClassifier(
    random_state=42,
    min_samples_leaf=5)
tr

DecisionTreeClassifier(min_samples_leaf=5, random_state=42)

In [145]:
rf = ensemble.RandomForestClassifier(
    n_estimators=1000,
    min_samples_leaf=5,
    random_state=42)
rf

RandomForestClassifier(min_samples_leaf=5, n_estimators=1000, random_state=42)

In [146]:
gbt = ensemble.GradientBoostingClassifier(
    n_estimators=500,
    min_samples_leaf=5,
    random_state=42,
    warm_start=True)
gbt

GradientBoostingClassifier(min_samples_leaf=5, n_estimators=500,
                           random_state=42, warm_start=True)

In [147]:
lr_results = run_model(lr, x_train, np.ravel(y_train), scoring, cv)
get_means(lr_results)

{'fit_time average': 1.4652323722839355,
 'score_time average': 0.006616950035095215,
 'test_f1_weighted average': 0.4427647907797243,
 'test_accuracy average': 0.5429396700613424}

In [148]:
nb_results = run_model(nb, x_train, np.ravel(y_train), scoring, cv)
get_means(nb_results)

{'fit_time average': 0.024904489517211914,
 'score_time average': 0.0073804140090942385,
 'test_f1_weighted average': 0.4362588545069312,
 'test_accuracy average': 0.5170695610743851}

In [149]:
knn_results = run_model(knn, x_train, np.ravel(y_train), scoring, cv)
means = get_means(knn_results)
means

{'fit_time average': 0.014935541152954101,
 'score_time average': 0.1542067289352417,
 'test_f1_weighted average': 0.518083816114842,
 'test_accuracy average': 0.5626031802751474}

In [150]:
tr_results = run_model(tr, x_train, np.ravel(y_train), scoring, cv)
get_means(tr_results)

{'fit_time average': 0.0645158052444458,
 'score_time average': 0.005397200584411621,
 'test_f1_weighted average': 0.5374550438161237,
 'test_accuracy average': 0.5707060329938658}

In [151]:
rf_results = run_model(rf, x_train, np.ravel(y_train), scoring, cv)
get_means(rf_results)

{'fit_time average': 8.890981912612915,
 'score_time average': 0.2961680173873901,
 'test_f1_weighted average': 0.5106039122326846,
 'test_accuracy average': 0.5765728664165326}

In [152]:
gbt_results = run_model(gbt, x_train, np.ravel(y_train), scoring, cv)
get_means(gbt_results)

{'fit_time average': 67.0042823791504,
 'score_time average': 0.04498279094696045,
 'test_f1_weighted average': 0.5439888469128278,
 'test_accuracy average': 0.5786412363766305}